In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk import pos_tag
from collections import Counter
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import os
import zipfile
import json
import random
import datetime as dt
import calendar

%run functions-prez.ipynb

In [2]:
# load data
ZIPFILE_NAME='annacalla14-batch-644-submitted-2019-04-05-21.53.zip'
ZIP_FILES = [f for f in os.listdir('.') if f.endswith('.zip')]
if not os.path.exists('data'):
    os.makedirs('data/text')

In [3]:
# sort/organize data
manifest_data=[]

text_file_cnt=0

for ZIPFILE_NAME in ZIP_FILES:

    with zipfile.ZipFile(ZIPFILE_NAME) as zf:
        for f in zf.filelist:
            if f.filename.count('plaintext')>0:
                fn=os.path.basename(f.filename)
                print('Extracting', fn)
                with open(os.path.join('data','text',fn),'wb') as out:
                    zipfile.shutil.copyfileobj(zf.open(f),out)
                    text_file_cnt+=1

            if f.filename.endswith('.csv'):
                mdf = pd.read_csv(zf.open(f))
                manifest_data.append(mdf)

mdf=pd.concat(manifest_data)
mdf.to_json('data/corpus_index.json', orient='records')

Extracting the-new-york-times-brett-kavanaugh_-the-4b265d24-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-judge-says-that-4b08f68a-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-ryan-seacrest-interviews-4aee6ac2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-suit-tests-instagram-49ee40a2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-weinstein-hires-a-49b4971c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-michael-avenatti’s-biggest-4ac54c96-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-jewish-crossroads-4a030776-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-silence-of-4967c856-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-cosby-verdict-is-498b5910-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-u.n.-cases-read-4afb8f0e-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-of-vampires-and-4a3f6522-58f1-11e9-a271-0242ac130002.txt
Extra

Extracting the-new-york-times-yes_-it-happens-4b027ddc-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-silent-abuse-of-49764e80-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-on-africa-trip_-49cad0fe-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-activists-exploit-effort-4a9b9694-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-at-retrial_-prosecutors-4a901288-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-mormon-women_s-group-4aa296c4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-meryl-streep_-kate-49cb8684-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-les-moonves_-fired-4b51d42c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-cancellation-of-4a3cf738-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-hooray-for-harvey-less-4b00ca14-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-_bretttoo_-49d755b8-58f1-11e9-a271-0242ac130002.txt
Extractin

Extracting the-new-york-times-tackling-sexual-harassment-4af3adac-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-judge_s-accuser-open-4a40e488-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-after-a-long-497ecc2c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-a-reckoning-for-4ad02f94-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-indian-cabinet-minister-49ef7eea-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-schneiderman-will-not-4a17f2e4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-save-the-phony-4aba5dd6-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-a-migrant-describes-4b317628-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-a-toxic-mix_-49f573a4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-r.-kelly-faces-4a8382ac-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-what-teenagers-think-4b398f7a-58f1-11e9-a271-0242ac130002.txt
Extracting t

Extracting the-new-york-times-harvey-weinstein-gets-4b3f778c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-in-cosby-and-4a28bd5e-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-before-_metoo_-there-49b79cd2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-harvey-weinstein’s-arrest-49c2519a-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-ontario’s-opposition-leader-4af1dd56-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-hillary-clinton’s-master-4aa3454c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-dealbook-briefing_-the-4b152fc2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-jamie-lee-curtis_-49e169ea-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-jewish-crossroads-4a2e5106-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-weinstein_s-request-for-4a32e798-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-why-did-hillary-49e58e9e-58f1-11e9-a27

Extracting fox-news-network-virginia-democrats-rocked-4bd4a2ee-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-the-left-in-4bf8c688-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-franken-faces-sexual-4c458f2c-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-cnn-is-going-4b9fae04-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-democrats-and-liberal-4b9a1b88-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-second-woman-comes-4ca85e72-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-california-psychology-professor-4baa5a84-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-the-real-danger-4be6686c-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-the-_me-too_-4c4b0d62-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-democrats-calling-for-4c953126-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-interview-with-sarah-4cbc8b9a-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-netw

...

Extracting the-new-york-times-bernie-sanders-apologizes-5c29b544-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-gender-war-5a702c9c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-r.-kelly-sued-57235226-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-50-more-women-5b3a8abe-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-raging-against-empty-59e3e7e6-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-weinstein-in-handcuffs-59348986-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-former-lieutenant-at-5b47de80-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-top-executive-leaves-5a9061e2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-korean-political-star-586287e2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-many-accusations_-few-58970fe4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-a-supreme-violation-5b71182c-58f1-11e9-a271-0242ac130002.txt


Extracting the-new-york-times-let-women-be-5838a1fc-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-save-the-phony-5a58aeb4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-from-russia-with-589645b4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-conversation_-seven-5891a0d6-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-consent-in-the-5996c9ca-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-brooklyn-rape-victim-57a8be16-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-cuomo-orders-a-585cf458-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-dealbook-briefing_-the-5c19c350-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-the-_metoo-movement-59f2660e-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-louis-c.k.-mocks-5b1a9cfe-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-_metoo-shakes-the-5a2e5ee8-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-yo

Extracting the-new-york-times-ontario_s-opposition-leader-5bd98cf4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-_metoo-makes-czech-59a5e25c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-women-describe-struggle-57162254-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-heitkamp-rebukes-senate-5c2c5100-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-r.-kelly-has-572a2ea2-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-prosecutor-won_t-charge-570f5168-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-susan-collins_-standing-58658668-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-top-dutch-orchestra-5a792536-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-at-sentencing-hearing_-5ad8cc7a-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-how-to-make-59ecb858-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-for-egypt_-a-5bd42fe8-58f1-11e9-a271-0242ac130002

Extracting the-new-york-times-11-of-our-5aac2a12-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-syria_-_metoo_-vikings_-57992de8-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-kavanaugh_s-fate-rests-58c2d6a6-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-dealbook-briefing_-how-5867d45e-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-facebook-calms-concerns-57a0c5a8-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-bill-cosby’s-lawyer-5bd1fe9e-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-_tony-was-a-572b92c4-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-woody-allen-sues-581ff03a-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-nbc-news-and-568dda66-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-education-secretary-betsy-5b4fe01c-58f1-11e9-a271-0242ac130002.txt
Extracting the-new-york-times-has-_metoo-gone-58af3588-58f1-11e9-a271-0242ac130002.txt
Extracting 

Extracting fox-news-network-google-ceo-to-5dc76388-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-is-mueller_s-probe-5e64c772-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-interview-with-sarah-5e170fbe-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-bill-clinton-questioned-5e630bb2-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-interview-with-kellyanne-5dcdc692-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-fmr-trump-aide-5d2a8d2e-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-california-psychology-professor-5e1dd948-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-second-woman-comes-5de3c00a-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-interview-with-doris-5ddf0678-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-christine-blasey-ford-5e7fad76-58f1-11e9-a271-0242ac130002.txt
Extracting fox-news-network-president-trump-holds-5dfc0714-58f1-11e9-a271-0242ac130002.txt
Extract

In [4]:
corpus = json.load(open('data/corpus_index.json'))
print('Corpus has', len(corpus), 'documents')

Corpus has 6737 documents


In [5]:
corpus[0]

{'Filename': 'the-new-york-times-hearing-set-for-4958142e-58f1-11e9-a271-0242ac130002.txt',
 'Publication': 'The New York Times',
 'Section': 'US; politics',
 'Date': '2018-09-17',
 'Title': 'Hearing Set for Monday to Hear Kavanaugh and His  Accuser',
 'Author': 'Sheryl Gay Stolberg and Julie Hirschfeld Davis',
 'LNID': '02A6A252C52394AB97B14672E56C2F2F53468850A86AD8BC534837F7C9F7357EEC6475A7810729DD190ACCBF85F1AD407673C48680DCAF8F2F020D1457CE182DCF4D8609C8B271CCB8926AF9F1DB8FF9C45D84D980EEB0A36B8BEB5732BEB6B1C414CD5D347A9C3A95A5888E4C074061'}